In [ ]:
# Import the pipeline
from cubediff_pipeline import create_cubediff_pipeline, generate_panorama

# Create the pipeline with the correct parameters
pipeline = create_cubediff_pipeline(
    pretrained_model_id="runwayml/stable-diffusion-v1-5",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# Generate a panorama
result = generate_panorama(
    prompt="A beautiful mountain landscape at sunset with a lake in the foreground",
    negative_prompt="blurry, ugly, distorted, low quality, low resolution, text, watermark",
    num_inference_steps=100,  # More steps for better quality
    guidance_scale=8.5,
    seed=42,
    pipeline=pipeline
)

# Display the panorama
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 7.5))
plt.imshow(result["panorama"])
plt.title(f"Generated Panorama")
plt.axis('off')
plt.show()

# Display individual cubemap faces
face_names = ['Front', 'Right', 'Back', 'Left', 'Top', 'Bottom']
fig, axes = plt.subplots(2, 3, figsize=(10, 6))
axes = axes.flatten()

for i, (face, name) in enumerate(zip(result["faces"], face_names)):
    axes[i].imshow(face)
    axes[i].set_title(name)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from tqdm import tqdm
from diffusers import DDIMScheduler, PNDMScheduler, DPMSolverMultistepScheduler


In [ ]:
from cubediff_models import load_sd_components, convert_attention_modules
from cubediff_utils import add_cubemap_positional_encodings, cubemap_to_equirect
from cubediff_inference import CubeDiffInference
from cubediff_pipeline import create_cubediff_pipeline

In [ ]:
# Load model components
vae, text_encoder, tokenizer, unet = load_sd_components(
    model_id="runwayml/stable-diffusion-v1-5", 
    use_sync_gn=True
)

# Convert UNet's attention to inflated attention
print("Converting UNet attention modules...")
unet = convert_attention_modules(unet)

# Set up scheduler (PNDM for better quality)
scheduler = PNDMScheduler.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    subfolder="scheduler"
)

In [ ]:
# Initialize the pipeline with enhanced settings
# pipeline = create_cubediff_pipeline(
#     pretrained_model_id="runwayml/stable-diffusion-v1-5",
#     scheduler_type="pndm",  # Try "pndm", "ddim", or "dpm"
#     use_enhanced_attention=True,
#     device="cuda" if torch.cuda.is_available() else "cpu"
# )

# Create inference pipeline
pipeline = CubeDiffInference(
    vae=vae,
    unet=unet,
    text_encoder=text_encoder,
    tokenizer=tokenizer,
    scheduler=scheduler,
    device="cuda" if torch.cuda.is_available() else "cpu"
)


In [ ]:
# Set parameters for generation with reduced memory usage
prompt = "A beautiful mountain landscape at sunset with a lake in the foreground"
negative_prompt = "blurry, ugly, distorted, low quality, low resolution, bad anatomy, worst quality, unrealistic, text, watermark"

# Generate the panorama with memory-optimized settings
result = pipeline.generate(
    prompt=prompt,
    negative_prompt=negative_prompt,
    height=512,  # Reduced from 768 to save memory
    width=512,   # Reduced from 768 to save memory
    num_inference_steps=75,  # Reduced from 100 to save memory
    guidance_scale=9.0,
    seed=42,
    return_faces=True,
    use_pos_encodings=True
)

In [ ]:
# Display the generated panorama
plt.figure(figsize=(15, 7.5))
plt.imshow(result["panorama"])
plt.title(f"Generated Panorama: {prompt}")
plt.axis('off')
plt.show()

In [ ]:
# Display individual cubemap faces
face_names = ['Front', 'Right', 'Back', 'Left', 'Top', 'Bottom']
fig, axes = plt.subplots(2, 3, figsize=(10, 6))
axes = axes.flatten()

for i, (face, name) in enumerate(zip(result["faces"], face_names)):
    axes[i].imshow(face)
    axes[i].set_title(name)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Try with different parameters
result_alt = pipeline.generate(
    prompt=prompt,
    negative_prompt=negative_prompt,
    height=768,
    width=768,
    num_inference_steps=75,  # Fewer steps for faster generation
    guidance_scale=8.0,  # Different guidance scale
    seed=123,  # Different seed
    return_faces=True,
    use_pos_encodings=True
)


In [ ]:

# Display the alternative panorama
plt.figure(figsize=(15, 7.5))
plt.imshow(result_alt["panorama"])
plt.title(f"Alternative Panorama: {prompt}")
plt.axis('off')
plt.show()